In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as f
from pyspark.sql.functions import col, signum

spark = SparkSession.builder.master("local[1]").getOrCreate()


In [2]:
#Need to carry over from day to day
#Dollar bars
df=spark.read.parquet("AllCandlestick.parquet")
#df=df.withColumn("Ticker",f.lit("SPY"))
df.show(5)
print(df.count())

+------+-------+------+-------+-------+------+----------+------+
|Ticker|   Open|  High|    Low|  Close|Volume|      Time|Status|
+------+-------+------+-------+-------+------+----------+------+
|   AAT|  35.82| 35.82|  35.82|  35.82| 304.0|1583965200|    ok|
|    AB|  35.88| 35.88|  35.88|  35.88| 100.0|1582235820|    ok|
|  AAAU|  14.89| 14.89|  14.88|  14.88| 868.0|1568210640|    ok|
|   AAP|157.964|158.58|157.964|158.105|6226.0|1573846380|    ok|
|  AACG|   1.28|  1.34|   1.26|   1.34|5370.0|1591882500|    ok|
+------+-------+------+-------+-------+------+----------+------+
only showing top 5 rows

2437523


In [3]:
import sys
df=df.withColumn('DolExch',col('Close')*col('Volume'))
df.select("DolExch").distinct().show(5)
df=df.withColumn("CumDolExch",f.sum("DolExch").over(Window.partitionBy('Ticker').orderBy("Time").rowsBetween(-sys.maxsize,0)))
df.show()
df.select("CumDolExch").distinct().show(5)
df=df.withColumn('DolBars', col('CumDolExch')%(1E7))#Take data pt every 10M dollars exchanged per stock
df=df.withColumn('Mark',f.when(df.DolBars<=df.DolExch,1).otherwise(0))
df.select("Mark").distinct().show()

+-------------+
|      DolExch|
+-------------+
|      231.235|
|  1232713.635|
|   1312601.32|
|1.603866854E7|
|     33301.68|
+-------------+
only showing top 5 rows

+------+-----+-----+-----+-------+------+----------+------+------------------+-----------------+
|Ticker| Open| High|  Low|  Close|Volume|      Time|Status|           DolExch|       CumDolExch|
+------+-----+-----+-----+-------+------+----------+------+------------------+-----------------+
|   AAT|47.64|47.64|47.64|  47.64|1621.0|1562607000|    ok|          77224.44|         77224.44|
|   AAT| 47.8| 47.8| 47.8|   47.8| 200.0|1562607600|    ok|            9560.0|         86784.44|
|   AAT| 47.8|47.97|47.74|  47.97|3076.0|1562607900|    ok|         147555.72|        234340.16|
|   AAT|47.96|47.97|47.89|47.8925|2097.0|1562607960|    ok|       100430.5725|      334770.7325|
|   AAT|47.89|47.91|47.88|  47.88|1744.0|1562608020|    ok|          83502.72|      418273.4525|
|   AAT|47.83|47.83|47.82|  47.82| 800.0|1562608080|   

In [4]:
df=df.withColumn('CumMark',f.sum("Mark").over(Window.partitionBy('Ticker').orderBy("Time").rowsBetween(-sys.maxsize,0)))
df.show(20)
df.select("CumMark").distinct().show(10)

+------+-----+-----+-----+-------+------+----------+------+------------------+-----------------+-----------------+----+-------+
|Ticker| Open| High|  Low|  Close|Volume|      Time|Status|           DolExch|       CumDolExch|          DolBars|Mark|CumMark|
+------+-----+-----+-----+-------+------+----------+------+------------------+-----------------+-----------------+----+-------+
|   AAT|47.64|47.64|47.64|  47.64|1621.0|1562607000|    ok|          77224.44|         77224.44|         77224.44|   1|      1|
|   AAT| 47.8| 47.8| 47.8|   47.8| 200.0|1562607600|    ok|            9560.0|         86784.44|         86784.44|   0|      1|
|   AAT| 47.8|47.97|47.74|  47.97|3076.0|1562607900|    ok|         147555.72|        234340.16|        234340.16|   0|      1|
|   AAT|47.96|47.97|47.89|47.8925|2097.0|1562607960|    ok|       100430.5725|      334770.7325|      334770.7325|   0|      1|
|   AAT|47.89|47.91|47.88|  47.88|1744.0|1562608020|    ok|          83502.72|      418273.4525|      41

In [5]:
df=df.withColumn("Volume", f.sum("Volume").over(Window.partitionBy("CumMark").orderBy("Time").rowsBetween(-sys.maxsize,0)))

df=df.withColumn("Low",f.min("Low").over(Window.partitionBy("CumMark").orderBy("Time").rowsBetween(-sys.maxsize,0)))
df.show()
print(df.head())
df=df.withColumn("Open",f.first("Open").over(Window.partitionBy("CumMark").orderBy("Time").rowsBetween(-sys.maxsize,0)))
print(df.head())
df=df.withColumn("Close",f.last("Close").over(Window.partitionBy("CumMark").orderBy("Time").rowsBetween(-sys.maxsize,0)))
df=df.filter(df.Mark==1)

#Add target vector(2 for +change, 1 for no change 0 for -change)
df=df.withColumn("next_val", f.lead(col("Close"),1).over(Window.partitionBy('Ticker').orderBy(df["Time"])))#Need to add partitionBy()
df=df.withColumn("label", (col("next_val")-col("Close"))/col("Close"))
print(df.head())
df=df.select("Ticker","Time","Close","Volume","label")#.withColumn("Target", 100*(col("next_val")-col("Close"))/col("Close"))
#df=df.drop("next_val").withColumn("Target",1+signum(col("Target")))
df.head()

+------+--------+--------+------+--------+--------+----------+------+------------------+--------------------+------------------+----+-------+
|Ticker|    Open|    High|   Low|   Close|  Volume|      Time|Status|           DolExch|          CumDolExch|           DolBars|Mark|CumMark|
+------+--------+--------+------+--------+--------+----------+------+------------------+--------------------+------------------+----+-------+
|  AAPL|  199.31|  199.31|199.03|199.0433|177630.0|1562608020|    ok|    3.5356061379E7|    8.683384657028E8|8338465.7028000355|   1|     26|
|  ABBV| 71.5335| 71.5529| 71.45|   71.45|260144.0|1562612640|    ok|         5895625.3|2.5500461000989988E8| 5004610.009899884|   1|     26|
|  ABBV| 71.4555|   71.56| 71.42|   71.56|289360.0|1562612700|    ok|        2090696.96| 2.570953069698999E8| 7095306.969899893|   0|     26|
|  ABBV|   71.56|   71.56| 71.42|    71.5|322435.0|1562612760|    ok|         2364862.5| 2.594601694698999E8| 9460169.469899893|   0|     26|
|   AA

Row(Ticker='AAT', Time=1562607000, Close=47.64, Volume=789039.0, label=0.007136859781695976)

In [6]:
#Convert Ticker value to int to include in ML algo, dictionary present (tick_dict) to convert back if needed
tickers=df.select("Ticker").distinct().rdd.flatMap(lambda x: x).collect()

tick_dict = {val : str(idx + 1) for idx, val in enumerate(tickers)} 
print(tick_dict)

from pyspark.sql.types import IntegerType
df=df.replace(to_replace=tick_dict, subset=['Ticker'])

df.printSchema()
df=df.withColumn("Ticker",col("Ticker").cast(IntegerType()))
df.printSchema()

{'AAT': '1', 'ABMD': '2', 'AA': '3', 'AAPL': '4', 'ACA': '5', 'ABM': '6', 'AACG': '7', 'ABEO': '8', 'ABTX': '9', 'ABIO': '10', 'ABEV': '11', 'ABBV': '12', 'AAP': '13', 'AAON': '14', 'AAWW': '15', 'ABUS': '16', 'ABG': '17', 'AAXN': '18', 'ACAM': '19', 'ABCB': '20', 'AAOI': '21', 'A': '22', 'ABEQ': '23', 'AAAU': '24', 'AAL': '25', 'ABR': '26', 'AAXJ': '27', 'AB': '28', 'AAU': '29', 'ABT': '30', 'ABB': '31', 'ABC': '32', 'AC': '33', 'ACBI': '34', 'AAME': '35', 'ACAD': '36', 'ACAMU': '37', 'ACB': '38', 'AADR': '39', 'ACAMW': '40'}
root
 |-- Ticker: string (nullable = true)
 |-- Time: long (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- label: double (nullable = true)

root
 |-- Ticker: integer (nullable = true)
 |-- Time: long (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- label: double (nullable = true)



In [7]:
from pyspark.ml.feature import VectorAssembler
#Create input features
df_cols=df.columns
df_cols=[ elem for elem in df_cols if elem not in ["Time","label"]]
#Move features to a single vector
assembler=VectorAssembler(inputCols=df_cols,outputCol="features")
df=assembler.transform(df)
df=df.na.drop()
df=df.dropDuplicates()
print(df.count())
df.show(10)

152960
+------+----------+--------+---------+--------------------+--------------------+
|Ticker|      Time|   Close|   Volume|               label|            features|
+------+----------+--------+---------+--------------------+--------------------+
|     1|1568754000|   47.08|6260529.0|0.005203908241291...|[1.0,47.08,626052...|
|     2|1564696020| 201.321|2371757.0|-1.21696196621261...|[2.0,201.321,2371...|
|     2|1565219760|  190.38| 562916.0|0.001812165143397...|[2.0,190.38,56291...|
|     2|1567543800|  185.44|2732836.0|-0.01396678170836...|[2.0,185.44,27328...|
|     2|1572565800|206.9308| 514884.0|3.281290170434209...|[2.0,206.9308,514...|
|     2|1576887480|  162.93| 622715.0|-3.68256306389260...|[2.0,162.93,62271...|
|     2|1577997300|  172.26| 514360.0|-0.01079763148728...|[2.0,172.26,51436...|
|     2|1578431760|178.9018| 648462.0|0.009939531072353649|[2.0,178.9018,648...|
|     2|1578945360|   161.0| 536686.0|-6.21118012421795...|[2.0,161.0,536686.0]|
|     2|1581553320|  

In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder



featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(df)

# Split the data into training and test sets (last 20% held out for testing)
df=df.sort("Time")
trainingData=df.limit(int(df.count()*0.8))
maxTime=trainingData.agg({'Time':'max'}).collect()[0]
maxTime=maxTime["max(Time)"]
testData=df.filter(df.Time>maxTime)


# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures")

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])

paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [3, 10, 20]) \
                              .addGrid(rf.numTrees, [5, 10, 20, 30]) \
                              .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(),
                          numFolds=5)

# Train model.  This also runs the indexer.
model = crossval.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)




+--------------------+--------------------+--------------------+
|          prediction|               label|            features|
+--------------------+--------------------+--------------------+
|-9.39038871691447...|-3.36033391107591...|[4.0,282.71,44759.0]|
|-9.39038871691447...| 6.89775928944253E-5|[4.0,282.7005,288...|
|-9.39038871691447...|3.876540608869640...|[30.0,94.93,19520...|
|-2.36515629835535E-4|5.571030640669395E-4|[31.0,17.95,80195...|
|-9.39038871691447...|4.598189020939284E-4|[4.0,282.72,88605.0]|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [10]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("R Squared on test data = %g" % r2)

#rfModel = model.stages[1]
#print(rfModel)  # summary only


R Squared on test data = -0.000352045


In [ ]:
#r2=0.000213893 for SPY_candlestick random split 80/20%
#r2=-0.000156503 for SPY_candlestick with first 80% being train data, last 20% is test
#r2=9.93879e-05 for "" with first 80% being train data, use 5-fold CV

In [ ]:
#RMSE test> RMSE train -> likely overfitting the data

In [ ]:
ranFor=RandomForestRegressor(labelCol="label", featuresCol="features")
modelRf=ranFor.fit(trainingData)
print(modelRf.featureImportances) #[Ticker, Close, Volume]


In [ ]:
predictions=predictions.repartition(100)
predictions.write.parquet('SPYpred.parquet')

In [ ]:
print(maxTime)

In [ ]:
print(df.agg({"Time":'max'}).collect()[0])

In [ ]:
int(df.count()*0.8)

In [ ]:
df.count()

In [ ]:
testData.count()

In [ ]:
predictions.filter(predictions.prediction<0).count()

In [ ]:
predictions.show(100)

In [15]:
predictions.select("Ticker","Time", "Close", "Volume", "label", "prediction").show(5)

+------+----------+--------+---------+--------------------+--------------------+
|Ticker|      Time|   Close|   Volume|               label|          prediction|
+------+----------+--------+---------+--------------------+--------------------+
|     4|1588026780|  282.71|  44759.0|-3.36033391107591...|-9.39038871691447...|
|     4|1588026840|282.7005|  28885.0| 6.89775928944253E-5|-9.39038871691447...|
|    30|1588026900|   94.93| 195200.0|3.876540608869640...|-9.39038871691447...|
|    31|1588026900|   17.95|8019501.0|5.571030640669395E-4|-2.36515629835535E-4|
|     4|1588026900|  282.72|  88605.0|4.598189020939284E-4|-9.39038871691447...|
+------+----------+--------+---------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
#IDEA: Add length of bar as a feature